In [2]:
from essentia import *
from essentia.standard import *
import pandas
import os
import numpy as np
import pickle

In [7]:
list_of_files = os.listdir("./audio_files/")
num_files = list_of_files.__len__()

print('preparing extractor...')
extractor = FreesoundExtractor(profile='custom_extractor_profile.yaml')
features = extractor("./audio_files/" + list_of_files[0])
features = features[0]
output = pandas.DataFrame(index=range(num_files), columns=[features.descriptorNames()], dtype='float')
all_features = features.descriptorNames()

print('extracting features...')
for file in range(num_files):
    if list_of_files[file] == '.DS_Store':
        print('DS_Store item encountered & removed')
        os.remove(list_of_files[file])
    path = "./audio_files/" + list_of_files[file]
    features = extractor(path)
    features = features[0]

    for feat in range(len(all_features)):
        current_feat = str(all_features[feat])
        if isinstance(features[current_feat], float):
            output.iloc[file, feat] = features[str(all_features[feat])]
        else: 
            break
    begin_list = feat + 1
    for feat in range(begin_list, len(all_features)):
        current_feature = features[str(all_features[feat])]
        if (type(current_feature) != str) or (not current_feature_mean):
            current_feature_list = current_feature.tolist()
            current_feature_mean = np.mean(current_feature_list)
            if np.isnan(current_feature_mean):
                output.iloc[file, feat] = 0.0
            else:
                output.iloc[file, feat] = current_feature_mean
        else: break
    end_list = feat
    print('extracting file ' + str(file) + ' of ' + str(num_files))

output.dropna(axis=1, inplace = True)
output.to_pickle('test_files_features.pkl')

preparing extractor...
extracting features...
extracting file 0 of 23
extracting file 1 of 23
extracting file 2 of 23
extracting file 3 of 23
extracting file 4 of 23
extracting file 5 of 23
extracting file 6 of 23
extracting file 7 of 23
extracting file 8 of 23
extracting file 9 of 23
extracting file 10 of 23
extracting file 11 of 23
extracting file 12 of 23
extracting file 13 of 23
extracting file 14 of 23
extracting file 15 of 23
extracting file 16 of 23
extracting file 17 of 23
extracting file 18 of 23
extracting file 19 of 23
extracting file 20 of 23
extracting file 21 of 23
extracting file 22 of 23


In [6]:
output

,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,lowlevel.barkbands_crest.stdev,...,tonal.hpcp.var,rhythm.beats_position,rhythm.bpm_histogram,rhythm.bpm_intervals,rhythm.onset_times,tonal.chords_histogram,lowlevel.gfcc.cov,lowlevel.gfcc.icov,lowlevel.mfcc.cov,lowlevel.mfcc.icov
0,0.000440,3.217287,4.471879,7.694623,16.675312,26.510815,14.383987,11.263230,3.021271,7.396245,...,0.037784,0.441179,0.000,0.232200,0.208980,4.166667,217.335213,0.001443,307.969122,0.000316
1,0.002832,3.893541,6.249127,22.948893,45.250694,24.707232,8.867496,6.351522,3.268399,6.081696,...,0.053508,0.429569,0.004,0.243810,0.516644,4.166667,15.177261,0.002072,17.107595,0.002205
2,0.028809,3.541288,5.342830,9.307620,23.313036,17.699018,7.775355,6.410821,2.942402,4.050940,...,0.063671,0.483749,0.004,0.238005,0.505034,4.166667,24.816609,0.002438,20.500359,0.000571
3,0.930655,2.628133,4.148837,3.440253,8.512089,16.178598,11.718757,12.312572,5.407354,2.584555,...,0.039714,0.000000,0.000,0.000000,0.354104,4.166667,21.356950,0.004632,41.993459,0.003295
4,0.962779,2.886811,5.271498,4.980785,12.898100,18.741545,10.366531,10.195642,4.927362,2.659385,...,0.019690,0.557279,0.004,0.371519,0.011610,4.166667,18.849658,0.003589,43.617125,0.003969
5,0.633277,2.386010,3.825182,3.095840,6.078234,24.723467,17.308043,17.232176,11.055050,4.332688,...,0.008985,0.000000,0.000,0.000000,0.278639,4.166667,8.511438,0.002525,5.471143,0.002294
6,0.000007,2.254482,3.545446,8.003052,13.940703,23.308971,15.956903,17.747377,4.245487,5.000264,...,0.023536,0.441179,0.004,0.325079,0.232200,4.166667,110.203331,0.002086,67.927224,-271.435003
7,0.590568,2.115329,3.353901,11.074517,25.239658,24.140228,11.104068,6.925972,3.584895,5.924506,...,0.022396,0.000000,0.000,0.000000,0.510839,4.166667,31.477518,0.001477,45.526692,0.000791
8,0.002882,3.412704,5.335426,8.317774,20.370935,23.037249,9.758578,8.673444,2.340329,4.572547,...,0.053621,0.425699,0.004,0.232200,0.201240,4.166667,41.487458,0.002231,93.274832,0.000383
9,0.835792,2.175823,3.703075,3.897818,9.432106,14.890794,8.669652,8.238066,4.281106,2.592264,...,0.022392,0.632744,0.004,0.429569,0.441179,4.166667,9.012219,0.004094,27.983938,0.002312
